# core

> Core functionality for `onprem`

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All, LlamaCpp
import chromadb
import os
import argparse
import time
import warnings
import sys
from typing import Any, Dict, Generator, List, Optional, Tuple, Union


In [ ]:
#| export

from onprem import utils as U
DEFAULT_MODEL_URL = 'https://huggingface.co/TheBloke/Wizard-Vicuna-7B-Uncensored-GGML/resolve/main/Wizard-Vicuna-7B-Uncensored.ggmlv3.q4_0.bin'
DEFAULT_MODEL_NAME = os.path.basename(DEFAULT_MODEL_URL)
DEFAULT_EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

class LLM:
    def __init__(self, 
                 model_name=DEFAULT_MODEL_NAME,
                 n_gpu_layers:Optional[int]=None, 
                 max_tokens:int=512, 
                 n_ctx:int=2048, 
                 n_batch:int=1024,
                 mute_stream=False,
                verbose=False):
        """
        LLM Constructor
        
        **Args:**
        
        - *n_gpu_layers*: Number of layers to be loaded into gpu memory. Default is `None`.
        - *max_tokens*: The maximum number of tokens to generate.
        - *n_ctx*: Token context window.
        - *n_batch*: Number of tokens to process in parallel.
        - *mute_stream*: Mute ChatGPT-like token stream output during generation
        - *verbose*: Verbosity
        """
        self.model_name = model_name
        if not os.path.isfile(os.path.join(U.get_datadir(), model_name)):
            warnings.warn('The model {model_name} does not exist in {U.get_datadir()}. '+\
                          'Please execute LLM.download() to download it.')
        self.llm = None
        self.n_gpu_layers = n_gpu_layers
        self.max_tokens = max_tokens
        self.n_ctx = n_ctx
        self.n_batch = n_batch
        self.callbacks = [] if mute_stream else [StreamingStdOutCallbackHandler()]
        self.verbose = verbose
 
    @classmethod
    def download_model(cls, model_url=DEFAULT_MODEL_NAME, confirm=True, ssl_verify=True):
        """
        Download an LLM in GGML format supported by [lLama.cpp](https://github.com/ggerganov/llama.cpp).
        
        **Args:**
        
        - *model_url*: URL of model
        - *confirm*: whether or not to confirm with user before downloading
        - *ssl_verify*: If True, SSL certificates are verified. 
                        You can set to False if corporate firewall gives you problems.
        """
        datadir = U.get_datadir()
        model_name = os.path.basename(model_url)
        filename = os.path.join(datadir, model_name)
        confirm_msg = f"You are about to download the LLM {model_name} to the {datadir} folder. Are you sure?"
        if os.path.isfile(filename):
            confirm_msg = f'There is already a file {model_name} in {datadir}.\n Do you want to still download it?'
            
        shall = True
        if confirm:
            shall = input("%s (Y/n) " % confirm_msg) == "Y"
        if shall:
            U.download(model_url, filename, verify=ssl_verify)
        else:
            warnings.warn(f'{model_name} was not downloaded because "Y" was not selected.')
        return
    
    def ingest(self, 
               source_directory:str,
               embedding_model_name:str ='sentence-transformers/all-MiniLM-L6-v2',
               embedding_model_kwargs:dict ={'device': 'cpu'}
              ):
        """
        Ingests all documents in `source_folder` into vector database.
        Previously-ingested documents are ignored.

        **Args:**
        
        - *source_directory*: path to folder containing document store
        - *embedding_model*: name of sentence-transformers model
        - *embedding_model_kwargs*: arguments to embedding model (e.g., `{device':'cpu'}`)
        
        **Returns:** `None`
        """
        from onprem import ingest as I
        I.ingest_documents(source_directory, 
                           embedding_model_name=embedding_model_name,
                           embedding_model_kwargs=embedding_model_kwargs)
 
        
    def check_model(self):
        datadir = U.get_datadir()
        model_path = os.path.join(datadir, self.model_name)
        if not os.path.isfile(model_path):
            raise ValueError(f'The LLM model {self.model_name} does not appear to have been downloaded. '+\
                             f'Execute the download_model() method to download it.')
        return model_path
        
 
    def load_llm(self):
        model_path = self.check_model()
        
        if not self.llm or not self.qa:
            self.llm =  llm = LlamaCpp(model_path=model_path, 
                                       max_tokens=self.max_tokens, 
                                       n_batch=self.n_batch, 
                                       callbacks=self.callbacks, 
                                       verbose=self.verbose, 
                                       n_gpu_layers=self.n_gpu_layers, 
                                       n_ctx=self.n_ctx)    

        return self.llm
        
        
    def prompt(self, prompt):
        """
        Send prompt to LLM to generate a response
        """
        llm = self.load_llm()
        return llm(prompt)  

In [ ]:
show_doc(LLM.download_model)

---

[source](https://github.com/amaiya/onprem/blob/main/onprem/core.py#L61){target="_blank" style="float:right; font-size:smaller"}

### LLM.download_model

>      LLM.download_model (model_url='Wizard-
>                          Vicuna-7B-Uncensored.ggmlv3.q4_0.bin', confirm=True,
>                          ssl_verify=True)

Download an LLM in GGML format supported by [lLama.cpp](https://github.com/ggerganov/llama.cpp).

**Args:**

- *model_url*: URL of model
- *confirm*: whether or not to confirm with user before downloading
- *ssl_verify*: If True, SSL certificates are verified. 
                You can set to False if corporate firewall gives you problems.

In [ ]:
llm = LLM(model_name=DEFAULT_MODEL_NAME)

In [ ]:
if not os.path.isfile( os.path.join(U.get_datadir(), DEFAULT_MODEL_NAME) ):
    LLM.download_model(DEFAULT_MODEL_URL, confirm=False)

In [ ]:
assert os.path.isfile(os.path.join(U.get_datadir(), DEFAULT_MODEL_NAME))

In [ ]:
show_doc(LLM.prompt)

---

[source](https://github.com/amaiya/onprem/blob/main/onprem/core.py#L120){target="_blank" style="float:right; font-size:smaller"}

### LLM.prompt

>      LLM.prompt (prompt)

Send prompt to LLM to generate a response

In [ ]:
prompt = """Extract the names of people in the supplied sentences. Here is an example:
Sentence: James Gandolfini and Paul Newman were great actors.
People:
James Gandolfini, Paul Newman
Sentence:
I like Cillian Murphy's acting. Florence Pugh is great, too.
People:"""

In [ ]:
saved_output = llm.prompt(prompt)

llama.cpp: loading model from /home/amaiya/onprem_data/Wizard-Vicuna-7B-Uncensored.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 5407.72 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  = 1024.00 MB



Cillian Murphy, Florence Pugh

In [ ]:
#show_doc(LLM.ingest)

In [ ]:
#llm.ingest('./sample_data')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()